{
  "Pressure": 1003.9,
  "Temperature": 31.4,
  "Wind_speed": 0,
  "Light_intensity": 183,
  "Wind_direction": 7,
  "Humidity": 73.7,
  "Rain": 0
}

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
    .appName("WeatherStationStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2,org.postgresql:postgresql:42.2.23") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cb9f25cd-a4bf-422d-933a-69576e6636a1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.cod

In [3]:
# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("ERROR")

# Define the schema for the JSON messages
schema = StructType([
    StructField("Pressure", DoubleType(), True),
    StructField("Temperature", DoubleType(), True),
    StructField("Wind_speed", IntegerType(), True),
    StructField("Light_intensity", IntegerType(), True),
    StructField("Wind_direction", IntegerType(), True),
    StructField("Humidity", IntegerType(), True),
    StructField("Rain", IntegerType(), True)
])

In [11]:
# Read from Kafka as a stream
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "AIDHOOSTATION") \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", 100) \
    .load()

# Parse the Kafka messages
parsed_df = kafka_df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# Add a timestamp column for when the data was processed
processed_df = parsed_df.withColumn("processed_at", current_timestamp())

In [12]:
processed_df.show(5)

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

In [8]:
# Write the streaming data to PostgreSQL using the same style as your batch processing
def write_to_postgres(df, epoch_id):
    df.write \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/postgres") \
        .option("dbtable", "weather_station") \
        .option("user", "postgres") \
        .option("password", "Parijat789") \
        .option("driver", "org.postgresql.Driver") \
        .mode("append") \
        .save()
    print(f"Batch {epoch_id} written to PostgreSQL: {df.count()} records")

query = processed_df.writeStream \
    .foreachBatch(write_to_postgres) \
    .outputMode("append") \
    .trigger(processingTime="10 seconds") \
    .start()

# Keep the streaming job running
query.awaitTermination()

25/03/15 11:14:35 ERROR MicroBatchExecution: Query [id = 69edfc49-c0c5-46b0-b2de-e8173fe96591, runId = 4ebaf513-2d44-4e57-8ca5-541fbe410f24] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/kafka010/KafkaConfigUpdater
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$.kafkaParamsForDriver(KafkaSourceProvider.scala:643)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan.toMicroBatchStream(KafkaSourceProvider.scala:482)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.$anonfun$applyOrElse$4(MicroBatchExecution.scala:101)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:94)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution$$anonfun$1.applyOrElse(MicroBatchExecution.scala:79)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:48

StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: [id = 69edfc49-c0c5-46b0-b2de-e8173fe96591, runId = 4ebaf513-2d44-4e57-8ca5-541fbe410f24]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE